# Project - Image Classification

Hola !

Objective of this project is to implement the following models for the CIFAR100 image classification dataset.

### Importing packages 

In [1]:
import numpy as np
import keras
import tensorflow as tf
from keras.datasets import cifar100
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras import backend as K
from keras.preprocessing import image
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.models import Model

import warnings
warnings.filterwarnings('ignore')


Using TensorFlow backend.


### Load data

In [2]:
num_classes = 100

# input image dimensions
img_rows, img_cols = 32, 32

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# channels first: 3 (rgb images) x 32 x 32
if K.image_data_format() == 'channels_first':
    # number of examples, channels, rows, columns
    x_train = x_train.reshape(x_train.shape[0], 3, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 3, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    # 28 x 28 x 1
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)

# normalize data     
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


###  Training parameters

In [3]:
# Mention the training parameters such as epochs, learning rate, number of samples etc.
batch_size = 128
epochs = 2

## Training CNN for CIFAR100

In [4]:
# Design the model architecture here
K.clear_session()
model = Sequential()
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())  
model.add(Dense(512, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 16)        448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 64)          18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)             

### Compile

In [5]:
# Compile the model architecture here

model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 50000 samples, validate on 10000 samples
Epoch 1/2
50000/50000 [==============================] - 15s 309us/step - loss: 0.0497 - accuracy: 0.9900 - val_loss: 0.0453 - val_accuracy: 0.9901
Epoch 2/2
50000/50000 [==============================] - 16s 326us/step - loss: 0.0431 - accuracy: 0.9902 - val_loss: 0.0411 - val_accuracy: 0.9903


### Training

In [6]:
# Train the transfer learning model here
pretrained_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=[32,32, 3])
pretrained_model.trainable = False

model = tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation='softmax')
])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


### Testing

In [7]:
model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

model.fit(x_test, y_test, batch_size=512, epochs = 20,verbose = 1)

Epoch 1/20
10000/10000 [==============================] - 37s 4ms/sample - loss: 0.0647 - acc: 0.0415
Epoch 2/20
10000/10000 [==============================] - 35s 4ms/sample - loss: 0.0464 - acc: 0.1897
Epoch 3/20
10000/10000 [==============================] - 35s 3ms/sample - loss: 0.0381 - acc: 0.3057
Epoch 4/20
10000/10000 [==============================] - 34s 3ms/sample - loss: 0.0321 - acc: 0.4105
Epoch 5/20
10000/10000 [==============================] - 35s 3ms/sample - loss: 0.0277 - acc: 0.5079
Epoch 6/20
10000/10000 [==============================] - 35s 4ms/sample - loss: 0.0243 - acc: 0.5840
Epoch 7/20
10000/10000 [==============================] - 34s 3ms/sample - loss: 0.0215 - acc: 0.6429
Epoch 8/20
10000/10000 [==============================] - 35s 4ms/sample - loss: 0.0192 - acc: 0.6974
Epoch 9/20
10000/10000 [==============================] - 33s 3ms/sample - loss: 0.0173 - acc: 0.7398
Epoch 10/20
10000/10000 [==============================] - 32s 3ms/sample - loss: 

### Save

In [8]:
# Save the model to disk
model.save("my_model.h5") 

### Inference

### Further Fun



*   Experiment with different model architectures
*   Play with different parameters such as convolution size, pooling, padding, striding, epochs, dropout etc.
*   Train a Dense Neural Network as baseline and compare the performance

